In [7]:


from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import ast
import albumentations as A
from tqdm import tqdm



In [8]:
bbox_params = A.BboxParams(
    format= 'pascal_voc',
    min_area=1,
    min_visibility=0.5,
    label_fields=['class_name']
)


# Source: https://github.com/albumentations-team/albumentations_examples/blob/master/notebooks/example_bboxes.ipynb
aug = A.Compose([
    A.LongestMaxSize(max_size=1024),
    A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0,p=1.0),
    A.ShiftScaleRotate(shift_limit=.25, scale_limit=0.2, p=0.3),
    A.RandomSizedCrop((900, 1000), 1024, 1024, p=.2),
    A.HorizontalFlip(p=.5),
    A.Rotate(limit=30,p=.8),
    A.MultiplicativeNoise(p=.2),
    A.RGBShift(r_shift_limit=40, g_shift_limit=40, b_shift_limit=40, p=.3),
    A.Blur(blur_limit=25, p=.2),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.35,p=.5),
    A.HueSaturationValue(p=.3),
    A.GaussNoise(var_limit=(10, 50),mean=50, p=.2),
    A.OneOf([
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1)], p=0.2),
    A.OneOf([
        A.RandomBrightnessContrast(),
        A.RandomGamma(gamma_limit=[50,200], p=.2),
        A.ToGray()], p=0.3),
        A.NoOp(p=.04)
]
    , bbox_params=bbox_params)

aug_val = A.Compose([
    A.LongestMaxSize(max_size=1024),
    A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0,p=1.0),
    A.ShiftScaleRotate(shift_limit=.25, scale_limit=0.2, p=0.3),
    A.RandomSizedCrop((900, 1000), 1024, 1024, p=.2),
    A.HorizontalFlip(p=.5),
    A.Rotate(limit=30,p=.8),
    A.MultiplicativeNoise(p=.2),
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.35,p=.5),
]
    , bbox_params=bbox_params)
aug_test = A.Compose([
    A.LongestMaxSize(max_size=1024),
    A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=0,p=1.0),
    A.ShiftScaleRotate(shift_limit=.25, scale_limit=0.2, p=0.3),
    A.RandomSizedCrop((900, 1000), 1024, 1024, p=.2),
    A.HorizontalFlip(p=.5),
    A.Rotate(limit=30,p=.8),
]
    , bbox_params=bbox_params)  

In [9]:
def augment(aug_func, mode='train'):
    DATASET_PART = mode
    path_sign_language_letter_request = f'./asl_nojz/{DATASET_PART}/images'
    path_labels = f'./asl_nojz/{DATASET_PART}/labels/'
    DATASET_PATH = 'asl_nojz_augmented'
    IMAGES_PATH = f'{DATASET_PATH}/{DATASET_PART}/images'
    LABELS_PATH = f'{DATASET_PATH}/{DATASET_PART}/labels'
    os.makedirs(DATASET_PATH, exist_ok=True)
    os.makedirs(IMAGES_PATH, exist_ok=True)
    os.makedirs(LABELS_PATH, exist_ok=True)

    rows = []

    for filename in os.listdir(path_sign_language_letter_request+'/'):
        image = cv2.imread(path_sign_language_letter_request +'/' + filename)
        load_bboxes = np.genfromtxt(path_labels+filename[:-4] + '.txt')
        load_bboxes = list(load_bboxes)
        bbox_cat = int(load_bboxes[0])
        bboxes = load_bboxes[1:]
        img_ht = int(image.shape[0])
        img_wd = int(image.shape[1])
        bb_width = int(round(bboxes[2] * image.shape[1], 0))
        bb_height = int(round(bboxes[3] * image.shape[0], 0))

        x_min = int((img_wd * bboxes[0]) - (bb_width/2))
        x_max = int((img_wd * bboxes[0]) + (bb_width/2))
        y_min = int(img_ht * bboxes[1] - (bb_height/2))
        y_max = int(img_ht * bboxes[1] + (bb_height/2))
        new_bboxes = [x_min, y_min, x_max, y_max]

        
        # Creating 25 augmented images to compensate for the small dataset
        for i in range(25):
            class_name = bbox_cat
            augmented = aug_func(image=image, bboxes=[new_bboxes], class_name=[bbox_cat])
            img_name = f'{filename[:-4]}_{i}.jpg'
            label_name = f'{filename[:-4]}_{i}.txt'
            for bbox in augmented['bboxes']:
                x_min, y_min, x_max, y_max = map(lambda v: int(v), bbox)
                
                # Converting back to Yolo format
                x_center = (x_min + x_max) / 2
                x_center /= 1024
                y_center = (y_min + y_max) / 2
                y_center /= 1024
                w = (x_max - x_min) / 1024
                h = (y_max - y_min) /1024
                new_bbox = [bbox_cat, x_center, y_center, w, h]
                # rows.append({
                #     'image_id': f'{file_name}',
                #     'bbox': new_bbox
                # })
            cv2.imwrite(f'{IMAGES_PATH}/{img_name}', augmented['image'])
            with open(f"{LABELS_PATH}/{label_name}", 'w') as f:
                f.write(f"{' '.join(str(x) for x in new_bbox)}")


    # Appending bbox data to new annotation.csv file 
    # pd.DataFrame(rows).to_csv(f'{DATASET_PATH}/annotations.csv', header=True, index=None)

    os.system("notify-send 'Done'")


In [10]:
augment(aug, mode='train')
augment(aug_val, mode='valid')
augment(aug_test, mode='test')